# 6.0 Modelling Prophet


Work book summary:

This work book takes the EUR-USD data and the pattern matches and uses Prophet model to predict the price of the next few time frames.

The results were:

Marubuzo:
+ Accuracy:	    0.5
+ Precision:    0.63
+ Recall:       0.54

Fractals:
+ Accuracy:	    0.44
+ Precision:	0.6
+ Recall:		0.48

This shows it wasn't as accurate as ARIMAX. Therefore I will use the Arimax as the the machine learning signal I will feed into the MLR models as a feature.


---

# Contents

- [1.0 About Prophet Model](#1.0-About-Prophet-Model)
- [2.0 Marubozu](#2.0-Marubozu])
    - [2.1 Load Data](#2.1-Load-Data)
    - [2.2 Model](#2.2-Model)
    - [2.3 Train/Test Split](#2.3-Train/Test-Split)
    - [2.4 Fit Model](#2.4-Fit-Model)
    - [2.5 Calculate Results](#2.5-Calculate-Results)
    - [2.6 Run Model](#2.6-Run-Model)
    - [2.7 Results](#2.7-Results)
- [3.0 Fractals](#3.0-Fractals)
    - [3.1 Load Data](#3.1-Load-Data)
    - [3.2 Model](#3.2-Model)
    - [3.3 Train/Test Split](#3.3-Train/Test-Split)
    - [3.4 Fit Model](#3.4-Fit-Model)
    - [3.5 Calculate Results](#3.5-Calculate-Results)
    - [3.6 Run Model](#3.6-Run-Model)
    - [3.7 Results](#3.7-Results)



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import calendar

from sklearn.metrics import r2_score, mean_squared_error
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

from fbprophet import Prophet
from statsmodels.tools.eval_measures import rmse


In [2]:
pd.set_option('display.max_columns', None)

---

# 1.0 Prophet Model

Prophet is an open source model from facebook. It is used to predict the yearly, weekly, and daily seasonality, plus holiday effects. 

It can have external variables but thesE need to be known for the future forecasts as well. For example if wendnesday is always the busiest day you can put that into the model. However not something like gold price as the future price is not known.

It was built for facebooks internal use originally and not with the currency markets in mind. However it will be interesting to try a new model and see how it performs with this task.

[Facebook Prophet](https://facebook.github.io/prophet/)

---

# 2.0 Marubozu

## 2.1 Load Data

In [3]:
# read in the daily data
daily = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/data/resampled/eur-usddailyMarubozu.csv', 
                    index_col='date', parse_dates=True)

In [4]:
# read in the pattern dates to be used to create the train and test data.
daily_pattern = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/patterns/dailyMarubozu.csv', 
                           parse_dates=True)

In [5]:
daily_pattern['pattern_end'] = pd.to_datetime(daily_pattern['pattern_end'])

In [6]:
# check format
daily_pattern.loc[1]

pattern_end   2000-10-20
Name: 1, dtype: datetime64[ns]

In [7]:
# check number of patterns
len(daily_pattern)

64

---

# 2.2 Model

## 2.3 Train/Test Split

In [8]:
# Create the test train split from the pattern date

def create_train_test_split(date, time_series, model_info):

    # Get index of pattern and add 6 (so 5) extra rows for Test/train set
    test_end_loc = time_series.index.get_loc(date) + 6
    
    # Create train/test set using index loc of pattern 
    train_test = time_series.iloc[:test_end_loc]
    
    # Set target values
    target_value = time_series.loc[time_series.index == date,'exit_price'].item()
    
    # add target price to dataset
    train_test.insert(0, 'target_price', target_value)
    
    # Add Signal so it can be determined whether we expect the price to go up or down.
    model_info['signal'] = time_series.loc[date,'marubozu']
    
    # insert the signal to dataset
    train_test.insert(0, 'signal', model_info['signal'])
    
    # create start and end points for the test/train splits
    model_info['start'] = len(train_test)-5
    model_info['end'] = len(train_test)-0
    
    # create the train and data sets
    model_info['train'] = train_test.iloc[:model_info['start']]
    model_info['test'] = train_test.iloc[model_info['start']:]

    return model_info

## 2.4 Fit Model

In [9]:
# Fit the model

def fit_prophet(df):
    
    # Instantiate the model
    m = Prophet()
    new_df = pd.DataFrame()
    
    # If the model is signalling a market fall use 'low' as the comparison price
    if df['signal'] == -1:
        new_df = df['train'][['low']]
        new_df.reset_index(level=0, inplace=True)
        new_df.rename(columns={'date':'ds','low':'y'}, inplace=True)
    else:
    # If the model is signalling a market rise use 'high' as the comparison price
        new_df = df['train'][['high']]
        new_df.reset_index(level=0, inplace=True)
        new_df.rename(columns={'date':'ds','high':'y'}, inplace=True)
   
    # fit the model
    m.fit(new_df) 
        
    # return model and predictions dataframe
    return m, m.make_future_dataframe(periods=5, freq='B')
    

## 2.5 Calculate Results

In [10]:
# Determine whether the signal is correct and the future price is surpassed
def meet_threshold(row):
    
    if row['signal'] == -1 and row['low'] <= row['target_price']:
        return -1
    elif row['signal'] == 1 and row['high'] >= row['target_price']:
        return 1    
    else:
        return 0

In [11]:
# Calculate whether the machine learning model predicted correctly

def ml_decision(row):
    if row['direction'] == -1 and row['preds'] <= row['target_price']:
        return -1
    elif row['direction'] == 1 and row['preds'] >= row['target_price']:
        return 1    
    else:
        return 0

In [12]:
# create a dateframe of outcomes

def create_results_outcomes_dataframe(test, predictions):    
    outcomes = pd.DataFrame()
    outcomes['low'] = test['low']
    outcomes['high'] = test['high']
    outcomes['preds'] = predictions.values
    outcomes['target_price'] = test['target_price']
    outcomes['direction'] = test['signal']
    outcomes['correct_call'] = test.apply(meet_threshold, axis=1)

    return outcomes

In [13]:
# function to print graph showing outcomes

def print_chart(outcomes):
    if model_info['signal'] == -1:
        outcomes['low'].plot(legend=False, figsize=(12,8))
    else:
        outcomes['high'].plot(legend=False, figsize=(12,8))

    outcomes['preds'].plot(legend=False);
    outcomes['target_price'].plot(legend=False);

In [14]:
# Calculate the results dataframe for RMSE ans MSE
def get_results(model_info,predictions):
        
    if model_info['signal'] == -1:
        mse = mean_squared_error(model_info['test']['low'], predictions)
        rmse_res = rmse(model_info['test']['low'], predictions)
    else:
        mse = mean_squared_error(model_info['test']['high'], predictions)
        rmse_res = rmse(model_info['test']['high'], predictions)       
    
    return rmse_res, mse

In [15]:
# Determine the classfication of the machine learning for the confusion matrix

def classify(outcomes):
    
    if max(outcomes['direction']) == 1:
        
        if max(outcomes['correct_call']) == 0 and max(outcomes['ml_correct_call']) == 0:
            return 'tn'
        elif max(outcomes['correct_call']) == 1 and max(outcomes['ml_correct_call']) == 1:
            return 'tp'
        elif max(outcomes['correct_call']) == 0 and max(outcomes['ml_correct_call']) == 1:
            return 'fp'
        elif max(outcomes['correct_call']) == 1 and max(outcomes['ml_correct_call']) == 0:
            return 'fn'
        
    elif max(outcomes['direction']) == -1:
        
        if min(outcomes['correct_call']) == 0 and min(outcomes['ml_correct_call']) == 0:
            return 'tn'
        elif min(outcomes['correct_call']) == -1 and min(outcomes['ml_correct_call']) == -1:
            return 'tp'
        elif min(outcomes['correct_call']) == 0 and min(outcomes['ml_correct_call']) == -1:
            return 'fp'
        elif min(outcomes['correct_call']) == -1 and min(outcomes['ml_correct_call']) == 0:
            return 'fn'
        
    else:
        return 'ERROR'
    

In [16]:
# add the train and test set prices for to compare against the predictions
def append_actual_results(sig, train, test):
    
    if sig == 1:
        return train['high'].append(test['high'], ignore_index=True)
    else:
        return train['low'].append(test['low'], ignore_index=True)


## 2.6 Run Model

In [17]:
# Main function to drive the model

# Dictionary to store the model data in
# model_info = {"train":None,"test":None,"start":None,"end":None,"signal":None}

# List to store the models results in
prophet_results = []

# loop through the matching patterns
for match in daily_pattern['pattern_end']:
    
    # Dictionary to store the model data in
    model_info = {"train":None,"test":None,"start":None,"end":None,"signal":None}

    # dictionary to store the iterations results
    results_dict = {'name':None,'pattern':None,'date':None,
                   'time_frame':None,'RMSE':None,
                   'MSE':None, 'classification':None}
    
    # Add model details
    results_dict['name'] = 'Prophet ' + str(match)
    results_dict['strategy'] = 'Prophet'
    results_dict['time_frame'] = 'daily'
    
    # create the test train split
    model_info = create_train_test_split(match, daily, model_info)
    
    # Only use a model with at least 30 values
    if len(model_info['train']) < 30:
        continue

    # fit the model and store the results
    model, future = fit_prophet(model_info)
        
    # create the predictions
    forecast = model.predict(future)
    
    # store the signal to use to properly forecast
    sig = model_info['train'].loc[model_info['train'].index == match, 'signal'].item()
    
    # add actual results to forecasts for comparison
    forecast['y'] = append_actual_results(sig, model_info['train'], model_info['test'])
        
    # Separate the predictions
    predictions = forecast['yhat'].tail(5)

    # Create outcomes to store model predictions outcomes dataframe 
    outcomes = create_results_outcomes_dataframe(model_info['test'], predictions)
    
    # add whether the machine learning model made the correct decision
    outcomes['ml_correct_call'] = outcomes.apply(ml_decision, axis=1)

    # add results and classifications to results dictionary
    results_dict['RMSE'], results_dict['MSE'] = get_results(model_info, predictions)
    results_dict['classification'] = classify(outcomes)

    # append results to the results list
    prophet_results.append(results_dict)



INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbp

---

# 2.7 Results

In [18]:
# Create a confusion matrix 
def create_cm(prophet_results):
    
    res_cm = [[0,0],
              [0,0]]
    
    for result in prophet_results:
        res = result['classification']
        
        if res == 'tp':
            res_cm[0][0] += 1
        elif res == 'fp':
            res_cm[0][1] += 1
        elif res == 'fn':
            res_cm[1][0] += 1
        elif res == 'tn':
            res_cm[1][1] += 1
    
    return res_cm

In [19]:
# Use the results and create a confusion matrix
cm = create_cm(prophet_results)

In [20]:
# create a dataframe with the results
cm_df = pd.DataFrame(cm, index=['pred_success', 'pred_non_success'], columns=['actual success', 'actual non_success'])
cm_df

,actual success,actual non_success
pred_success,22,13
pred_non_success,19,10


In [21]:
def print_metrics(cm):
    # Accuracy - how many did the model get right
    # Total number of correct predictions / total number of predictions
    acc= (cm[0][0]+cm[1][1])/(np.sum(cm))
    
    # Precision proportion of positive identifications that were actually correct
    # True positives/ true positives + false positives)
    prec = cm[0][0]/(cm[0][0]+cm[0][1])
    
    # Recall - proportion of actual positives that were correctly defined
    # True positives/ true positives + false negatives
    rec = cm[0][0]/(cm[0][0]+cm[1][0])

    print(f"Accuracy:\t{round(acc,2)}\nPrecision:\t{round(prec,2)}\nRecall:\t\t{round(rec,2)}")


In [22]:
# Display the results
print_metrics(cm)

Accuracy:	0.5
Precision:	0.63
Recall:		0.54


---

# 3.0 Fractals

## 3.1 Load Data

In [23]:
# read in the daily data
daily_fractals = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/data/resampled/eur-usddailyfractals.csv', 
                    index_col='date', parse_dates=True)

In [24]:
# read in the pattern dates to be used to create the train and test data.
fractal_pattern = pd.read_csv('/Users/stuartdaw/Documents/Capstone_data/patterns/dailyfractals.csv', 
                           parse_dates=True)

In [25]:
fractal_pattern['pattern_end'] = pd.to_datetime(fractal_pattern['pattern_end'])

In [26]:
# check format
fractal_pattern.loc[1]

pattern_end   2000-07-28
Name: 1, dtype: datetime64[ns]

In [27]:
# check number of patterns
len(fractal_pattern)

613

---

# 3.2 Model

## 3.3 Train/Test Split

In [28]:
# Create the test train split from the pattern date

def create_train_test_split(date, time_series, model_info):

    # Get index of pattern and add 6 (so 5) extra rows for Test/train set
    test_end_loc = time_series.index.get_loc(date) + 6
    
    # Create train/test set using index loc of pattern 
    train_test = time_series.iloc[:test_end_loc]
    
    # Set target values
    target_value = time_series.loc[time_series.index == date,'exit_price'].item()
    
    # add target price to dataset
    train_test.insert(0, 'target_price', target_value)
    
    # Add Signal so it can be determined whether we expect the price to go up or down.
    model_info['signal'] = time_series.loc[date,'fractal_end']
    
    # insert the signal to dataset
    train_test.insert(0, 'signal', model_info['signal'])
    
    # create start and end points for the test/train splits
    model_info['start'] = len(train_test)-5
    model_info['end'] = len(train_test)-0
    
    # create the train and data sets
    model_info['train'] = train_test.iloc[:model_info['start']]
    model_info['test'] = train_test.iloc[model_info['start']:]

    return model_info

## 3.4 Fit Model

In [29]:
# Fit the model

def fit_prophet(df):
    
    # Instantiate the model
    m = Prophet()
    new_df = pd.DataFrame()
    
    # If the model is signalling a market fall use 'low' as the comparison price
    if df['signal'] == -1:
        new_df = df['train'][['low']]
        new_df.reset_index(level=0, inplace=True)
        new_df.rename(columns={'date':'ds','low':'y'}, inplace=True)
    else:
    # If the model is signalling a market rise use 'high' as the comparison price
        new_df = df['train'][['high']]
        new_df.reset_index(level=0, inplace=True)
        new_df.rename(columns={'date':'ds','high':'y'}, inplace=True)
   
    # fit the model
    m.fit(new_df) 
        
    # return model and predictions dataframe
    return m, m.make_future_dataframe(periods=5, freq='B')
    

## 3.5 Calculate Results

In [30]:
# Determine whether the signal is correct and the future price is surpassed
def meet_threshold(row):
    
    if row['signal'] == -1 and row['low'] <= row['target_price']:
        return -1
    elif row['signal'] == 1 and row['high'] >= row['target_price']:
        return 1    
    else:
        return 0

In [31]:
# Calculate whether the machine learning model predicted correctly

def ml_decision(row):
    if row['direction'] == -1 and row['preds'] <= row['target_price']:
        return -1
    elif row['direction'] == 1 and row['preds'] >= row['target_price']:
        return 1    
    else:
        return 0

In [32]:
# create a dateframe of outcomes

def create_results_outcomes_dataframe(test, predictions):    
    outcomes = pd.DataFrame()
    outcomes['low'] = test['low']
    outcomes['high'] = test['high']
    outcomes['preds'] = predictions.values
    outcomes['target_price'] = test['target_price']
    outcomes['direction'] = test['signal']
    outcomes['correct_call'] = test.apply(meet_threshold, axis=1)

    return outcomes

In [33]:
# function to print graph showing outcomes

def print_chart(outcomes):
    if model_info['signal'] == -1:
        outcomes['low'].plot(legend=False, figsize=(12,8))
    else:
        outcomes['high'].plot(legend=False, figsize=(12,8))

    outcomes['preds'].plot(legend=False);
    outcomes['target_price'].plot(legend=False);

In [34]:
# Calculate the results dataframe for RMSE ans MSE
def get_results(model_info,predictions):
        
    if model_info['signal'] == -1:
        mse = mean_squared_error(model_info['test']['low'], predictions)
        rmse_res = rmse(model_info['test']['low'], predictions)
    else:
        mse = mean_squared_error(model_info['test']['high'], predictions)
        rmse_res = rmse(model_info['test']['high'], predictions)       
    
    return rmse_res, mse

In [35]:
# Determine the classfication of the machine learning for the confusion matrix

def classify(outcomes):
    
    if max(outcomes['direction']) == 1:
        
        if max(outcomes['correct_call']) == 0 and max(outcomes['ml_correct_call']) == 0:
            return 'tn'
        elif max(outcomes['correct_call']) == 1 and max(outcomes['ml_correct_call']) == 1:
            return 'tp'
        elif max(outcomes['correct_call']) == 0 and max(outcomes['ml_correct_call']) == 1:
            return 'fp'
        elif max(outcomes['correct_call']) == 1 and max(outcomes['ml_correct_call']) == 0:
            return 'fn'
        
    elif max(outcomes['direction']) == -1:
        
        if min(outcomes['correct_call']) == 0 and min(outcomes['ml_correct_call']) == 0:
            return 'tn'
        elif min(outcomes['correct_call']) == -1 and min(outcomes['ml_correct_call']) == -1:
            return 'tp'
        elif min(outcomes['correct_call']) == 0 and min(outcomes['ml_correct_call']) == -1:
            return 'fp'
        elif min(outcomes['correct_call']) == -1 and min(outcomes['ml_correct_call']) == 0:
            return 'fn'
        
    else:
        return 'ERROR'
    

In [36]:
# add the train and test set prices for to compare against the predictions
def append_actual_results(sig, train, test):
    
    if sig == 1:
        return train['high'].append(test['high'], ignore_index=True)
    else:
        return train['low'].append(test['low'], ignore_index=True)


## 3.6 Run Model

In [37]:
# Main function to drive the model

# Dictionary to store the model data in
# model_info = {"train":None,"test":None,"start":None,"end":None,"signal":None}

# List to store the models results in
prophet_results = []

# loop through the matching patterns
for match in fractal_pattern['pattern_end']:
    
    # Dictionary to store the model data in
    model_info = {"train":None,"test":None,"start":None,"end":None,"signal":None}

    # dictionary to store the iterations results
    results_dict = {'name':None,'pattern':None,'date':None,
                   'time_frame':None,'RMSE':None,
                   'MSE':None, 'classification':None}
    
    # Add model details
    results_dict['name'] = 'Prophet ' + str(match)
    results_dict['strategy'] = 'Prophet'
    results_dict['time_frame'] = 'daily'
    
    # create the test train split
    model_info = create_train_test_split(match, daily_fractals, model_info)
    
    # Only use a model with at least 30 values
    if len(model_info['train']) < 30:
        continue

    # fit the model and store the results
    model, future = fit_prophet(model_info)
        
    # create the predictions
    forecast = model.predict(future)
    
    # store the signal to use to properly forecast
    sig = model_info['train'].loc[model_info['train'].index == match, 'signal'].item()
    
    # add actual results to forecasts for comparison
    forecast['y'] = append_actual_results(sig, model_info['train'], model_info['test'])
        
    # Separate the predictions
    predictions = forecast['yhat'].tail(5)

    # Create outcomes to store model predictions outcomes dataframe 
    outcomes = create_results_outcomes_dataframe(model_info['test'], predictions)
    
    # add whether the machine learning model made the correct decision
    outcomes['ml_correct_call'] = outcomes.apply(ml_decision, axis=1)

    # add results and classifications to results dictionary
    results_dict['RMSE'], results_dict['MSE'] = get_results(model_info, predictions)
    results_dict['classification'] = classify(outcomes)

    # append results to the results list
    prophet_results.append(results_dict)



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations. Using 24.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override t

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


---

# 3.7 Results

In [38]:
# Create a confusion matrix 
def create_cm(prophet_results):
    
    res_cm = [[0,0],
              [0,0]]
    
    for result in prophet_results:
        res = result['classification']
        
        if res == 'tp':
            res_cm[0][0] += 1
        elif res == 'fp':
            res_cm[0][1] += 1
        elif res == 'fn':
            res_cm[1][0] += 1
        elif res == 'tn':
            res_cm[1][1] += 1
    
    return res_cm

In [39]:
# Use the results and create a confusion matrix
cm_fr = create_cm(prophet_results)

In [40]:
# create a dataframe with the results
cm_df_fr = pd.DataFrame(cm_fr, index=['pred_success', 'pred_non_success'], columns=['actual success', 'actual non_success'])
cm_df_fr

,actual success,actual non_success
pred_success,194,127
pred_non_success,214,74


In [41]:
def print_metrics(cm_fr):
    # Accuracy - how many did the model get right
    # Total number of correct predictions / total number of predictions
    acc= (cm_fr[0][0]+cm_fr[1][1])/(np.sum(cm_fr))
    
    # Precision proportion of positive identifications that were actually correct
    # True positives/ true positives + false positives)
    prec = cm_fr[0][0]/(cm_fr[0][0]+cm_fr[0][1])
    
    # Recall - proportion of actual positives that were correctly defined
    # True positives/ true positives + false negatives
    rec = cm_fr[0][0]/(cm_fr[0][0]+cm_fr[1][0])

    print(f"Accuracy:\t{round(acc,2)}\nPrecision:\t{round(prec,2)}\nRecall:\t\t{round(rec,2)}")


In [42]:
# Display the results
print_metrics(cm_fr)

Accuracy:	0.44
Precision:	0.6
Recall:		0.48
